In [1]:
#!usr/bin/env python

from __future__ import print_function

In [2]:
from pyrosetta import *
from rosetta import *

# tools
import sys, os
import commands
import random

# rosetta corrected imports
import rosetta.protocols.membrane
from rosetta.core.pose import Pose
from rosetta.core.scoring import ScoreFunction
from rosetta.core.pack.task import TaskFactory
from rosetta.utility import vector1_bool
from rosetta.core.chemical import aa_from_one_or_three
from rosetta.protocols.minimization_packing import PackRotamersMover
from rosetta.core.pose import PDBInfo
from rosetta.core.chemical import VariantType
from pyrosetta.toolbox import mutate_residue

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Import of 'rosetta' as a top-level module is deprecated and may be removed in 2018, import via 'pyrosetta.rosetta'.
  


In [3]:
init( extra_options="-mp:setup:spanfiles TCM_pR.span")
pose = pose_from_pdb( "TCM_pR.pdb" )

Found rosetta database at: /usr/local/lib/python2.7/dist-packages/pyrosetta-2018.12+release.5ecebca-py2.7-linux-x86_64.egg/pyrosetta/database; using it....
PyRosetta-4 2017 [Rosetta PyRosetta4.Release.python27.ubuntu 2018.12+release.5ecebca5ecebcadacdf48ede3c1981444601bf1cd47ce0d 2018-03-23T13:02:49] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.

core.init: Checking for fconfig files in pwd and ./rosetta/flags

core.init: Rosetta version: PyRosetta4.Release.python27.ubuntu r174 2018.12+release.5ecebca 5ecebcadacdf48ede3c1981444601bf1cd47ce0d http://www.pyrosetta.org 2018-03-23T13:02:49
core.init: command: PyRosetta -ex1 -ex2aro -mp:setup:spanfiles TCM_pR.span -database /usr/local/lib/python2.7/dist-packages/pyrosetta-2018.12+release.5ecebca-py2.7-linux-x86_64.egg/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=-497268600 seed_offset=0 real_seed=-497268600
c

In [4]:
add_memb = rosetta.protocols.membrane.AddMembraneMover()
add_memb.apply( pose )

protocols.membrane.AddMembraneMover: =====================================================================
protocols.membrane.AddMembraneMover: ||           WELCOME TO THE WORLD OF MEMBRANE PROTEINS...          ||
protocols.membrane.AddMembraneMover: =====================================================================
protocols.membrane.AddMembraneMover: No membrane residue was found
protocols.membrane.AddMembraneMover: Adding a new membrane residue to the pose
protocols.membrane.AddMembraneMover: Adding a membrane residue representing the position of the membrane after residue 235
protocols.membrane.AddMembraneMover:    Edge   	   Jump     Jump #
          	0236--0001  001
0001--0235
core.conformation.membrane.SpanningTopology: Filling membrane spanning topology from spanfile TCM_pR.span
core.conformation.membrane.SpanningTopology: nres: 236
core.conformation.membrane.SpanningTopology: nres_topo: 235
core.conformation.membrane.SpanningTopology: total residues in spanfile: 235
core.co

In [6]:
init_mem_pos = rosetta.protocols.membrane.MembranePositionFromTopologyMover()
init_mem_pos.apply( pose )

protocols.membrane.MembranePositionMoverFromTopologyMover: Reordering foldtree:
protocols.membrane.MembranePositionMoverFromTopologyMover: Starting foldtree: Is membrane fixed? 0
protocols.membrane.MembranePositionMoverFromTopologyMover:    Edge   	   Jump     Jump #
          	0001--0236  001
0001--0235
protocols.membrane.MembranePositionMoverFromTopologyMover: Computing initial membrane position from structure...
protocols.membrane.geometry.Embedding: Constructing Embedding object from topology and pose
protocols.membrane.geometry.Embedding: Computing membrane embedding from TMspans:
protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 8 to 35
protocols.membrane.geometry.EmbeddingDef: [ WARNING ] If your starting PDB is already translated into the
protocols.membrane.geometry.EmbeddingDef: [ WARNING ] membrane, then your TMspan does not span the membrane!
protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 8 to 35
protocols.

In [7]:
sfxn = create_score_function( "mpframework_smooth_fa_2012" )

core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/PairEPotential/pdb_pair_stats_fine
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
core.scoring.etable: Using alternate parameters: LK_DGFREE in MembEtable construction.
core.scoring.etable: Using alternate parameters: MEMB_LK_DGFREE in MembEtable construction.
core.

In [8]:
mutate_residue

<function pyrosetta.toolbox.mutate_residue>

In [12]:
repacked_native = mutate_residue( pose, 167, 'S', 8.0, sfxn )
AAs = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
with file( 'pR_ddG.out', 'a' ) as f: 
    f.write( "Res AA ddG\n" )
    for aa in AAs: 
        ddG = compute_ddG( repacked_native, sfxn, 181, aa, 8.0 )
        f.write( str(181) + " " + aa + " " + str( round( ddG, 3 ) ) + "\n" )

RuntimeError: 

File: /home/benchmark/T/rosetta.Glass/_commits_/main/source/src/core/scoring/membrane/FaMPEnvSmoothEnergy.cc:379
[ ERROR ] UtilityExitException
ERROR: ERROR: Failed to find amino acid UNK in EnvSmooth::representative_atom_name



In [6]:
## framework, calculate ddGs 
def main( argv ):

    ## Step 1: Initialize Rosetta, including spanfile for the membrane framework
    init( extra_options="-mp:setup:spanfiles TCM_pR.span" )

    ## Step 2: Load in pose as a membrane pose (using AddMembraneMover)
    pose = pose_from_pdb( "TCM_pR.pdb" );
    add_memb = rosetta.protocols.membrane.AddMembraneMover()
    add_memb.apply( pose )
    
    ## Step 3: Orient Pose in the membrane based on transmembrane spans 
    init_mem_pos = rosetta.protocols.membrane.MembranePositionFromTopologyMover()
    init_mem_pos.apply( pose )

    ## Step 4: Create a membrane energy function
    ## @note: Two possible energy functions: (1) standard membrane full atom energy function
    ## or (2) energy function accommodating for pH (requires extra flags - see README)
    sfxn = create_score_function( "mpframework_smooth_fa_2012" );

    ## Step 5: Repack neighbors within 8.0 angstroms of the mutant position
    repacked_native = mutate_residue( pose, 167, 'S', 8.0, sfxn )

    ## Step 6: Compute the ddG of mutation from alanine to each of the 20 canonical
    ## amino acids. Then print the ddG file to an output file (ompLA_ddG.out)
    AAs = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    with file( 'pR_ddG.out', 'a' ) as f: 
        f.write( "Res AA ddG\n" )
        for aa in AAs: 
            ddG = compute_ddG( repacked_native, sfxn, 181, aa, 8.0 )
            f.write( str(181) + " " + aa + " " + str( round( ddG, 3 ) ) + "\n" )
    f.close

In [7]:
## @brief Compute the ddG of mutation
## @brief Compute ddG of mutation in a protein at specified residue and AA position
def compute_ddG( pose, sfxn, resnum, aa, repack_radius ): 

    # Score Native Pose
    native_score = sfxn( pose )

    # Perform Mutation at residue <resnum> to amino accid <aa>
    mutated_pose = mutate_residue( pose, resnum, aa, repack_radius, sfxn )

    # Score Mutated Pose
    mutant_score = sfxn( mutated_pose )

    # Print resulting ddG
    return mutant_score - native_score


In [ ]:
# @brief Replace the residue at <resid> in <pose> with <new_res> and allows
# repacking within a given <pack_radius> 
def mutate_residue( pose, mutant_position, mutant_aa, pack_radius, sfxn ):

    if pose.is_fullatom() == False:
        IOError( 'mutate_residue only works with fullatom poses' )

    test_pose = Pose()
    test_pose.assign( pose )

    # Create a packer task (standard)
    task = TaskFactory.create_packer_task( test_pose )

    # the Vector1 of booleans (a specific object) is needed for specifying the
    #    mutation, this demonstrates another more direct method of setting
    #    PackerTask options for design
    aa_bool = vector1_bool()

    # PyRosetta uses several ways of tracking amino acids (ResidueTypes)
    # the numbers 1-20 correspond individually to the 20 proteogenic amino acids
    # aa_from_oneletter returns the integer representation of an amino acid
    #    from its one letter code
    # convert mutant_aa to its integer representation
    mutant_aa = core.chemical.aa_from_oneletter_code( mutant_aa )

In [ ]:
# make a new mutate_residue function
def mutate( pose, mutant_position, R1A_res, pack_radius, sfxn):
    if pose.is_fullatom() == False:
        IOError( 'mutate only works with fullatom poses')
    test_pose = Pose()
    test_pose.assign( pose )
    task = TaskFactory.create_packer_task( test_pose )
    R1A_pose = pose_from_sequence('Z[R1A]', res_type="fa_standard")
    R1A_res = R1A_pose.residue(1)
    
    
    
pose.replace_residue(167, R1A_res, True)

In [8]:
    # mutation is performed by using a PackerTask with only the mutant
    #    amino acid available during design
    # to do this, construct a Vector1 of booleans indicating which amino acid
    #    (by its numerical designation, see above) to allow
    for i in range( 1 , 21 ):
        # in Python, logical expression are evaluated with priority, thus the
        #    line below appends to aa_bool the truth (True or False) of the
        #    statement i == mutant_aa
        aa_bool.append( i == mutant_aa )

    # modify the mutating residue's assignment in the PackerTask using the
    #    Vector1 of booleans across the proteogenic amino acids
    task.nonconst_residue_task( mutant_position
        ).restrict_absent_canonical_aas( aa_bool )

    # prevent residues from packing by setting the per-residue "options" of
    #    the PackerTask
    center = pose.residue( mutant_position ).nbr_atom_xyz()
    for i in range( 1, pose.total_residue() + 1 ): 
        dist = center.distance_squared( test_pose.residue( i ).nbr_atom_xyz() );  
        # only pack the mutating residue and any within the pack_radius
        if i != mutant_position and dist > pow( float( pack_radius ), 2 ) :
            task.nonconst_residue_task( i ).prevent_repacking()

    # apply the mutation and pack nearby residues
    packer = PackRotamersMover( sfxn , task )
    packer.apply( test_pose )

    return test_pose

In [9]:
if __name__ == "__main__" : main(sys.argv)

Found rosetta database at: /usr/local/lib/python2.7/dist-packages/pyrosetta-2018.12+release.5ecebca-py2.7-linux-x86_64.egg/pyrosetta/database; using it....
PyRosetta-4 2017 [Rosetta PyRosetta4.Release.python27.ubuntu 2018.12+release.5ecebca5ecebcadacdf48ede3c1981444601bf1cd47ce0d 2018-03-23T13:02:49] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.

core.init: Checking for fconfig files in pwd and ./rosetta/flags

core.init: Rosetta version: PyRosetta4.Release.python27.ubuntu r174 2018.12+release.5ecebca 5ecebcadacdf48ede3c1981444601bf1cd47ce0d http://www.pyrosetta.org 2018-03-23T13:02:49
core.init: command: PyRosetta -ex1 -ex2aro -mp:setup:spanfiles TCM_pR.span -database /usr/local/lib/python2.7/dist-packages/pyrosetta-2018.12+release.5ecebca-py2.7-linux-x86_64.egg/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=-1767775646 seed_offset=0 real_seed=-1767775646

protocols.membrane.MembranePositionMoverFromTopologyMover: Computing initial membrane position from structure...
protocols.membrane.geometry.Embedding: Constructing Embedding object from topology and pose
protocols.membrane.geometry.Embedding: Computing membrane embedding from TMspans:
protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 8 to 35
protocols.membrane.geometry.EmbeddingDef: [ WARNING ] If your starting PDB is already translated into the
protocols.membrane.geometry.EmbeddingDef: [ WARNING ] membrane, then your TMspan does not span the membrane!
protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 8 to 35
protocols.membrane.geometry.EmbeddingDef: [ WARNING ] If your starting PDB is already translated into the
protocols.membrane.geometry.EmbeddingDef: [ WARNING ] membrane, then your TMspan does not span the membrane!
protocols.membrane.geometry.EmbeddingDef: Computing membrane embedding from TMspan 39 to 56
protocols

RuntimeError: 

File: /home/benchmark/T/rosetta.Glass/_commits_/main/source/src/core/scoring/membrane/FaMPEnvSmoothEnergy.cc:379
[ ERROR ] UtilityExitException
ERROR: ERROR: Failed to find amino acid UNK in EnvSmooth::representative_atom_name

